# Clustering Crypto

In [24]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [25]:
# Load the crypto_data.csv dataset.
crypto0_df = pd.read_csv("Data/crypto_data.csv")
crypto0_df.head(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [26]:
crypto0_df.dtypes

Unnamed: 0          object
CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [27]:
drop_index = crypto0_df.set_index(["Unnamed: 0"])
drop_index.index.name = None
drop_index.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [28]:
# Keep all the cryptocurrencies that are being traded.
crypto_isTrading_df = drop_index[drop_index["IsTrading"] != False]
crypto_isTrading_df.shape

(1144, 6)

In [29]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_workingAlg_df = crypto_isTrading_df[crypto_isTrading_df["Algorithm"] != 0]
print(crypto_workingAlg_df.shape)
crypto_workingAlg_df.head(10)

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [30]:
# Remove the "IsTrading" column.
crypto_workingAlg_df.drop(columns=["IsTrading"], inplace=True)
print(crypto_workingAlg_df.shape)
crypto_workingAlg_df.head(10)

(1144, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [31]:
# Count Null values
crypto_workingAlg_df.isnull().sum()

CoinName             0
Algorithm            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [32]:
# Remove rows that have at least 1 null value.
crypto_noNull_df = crypto_workingAlg_df.dropna(how='any')
print(crypto_noNull_df.shape)
crypto_noNull_df.head(10)

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [33]:
# Keep the rows where coins are mined.
crypto_mined_df = crypto_noNull_df[crypto_noNull_df["TotalCoinsMined"] > 0]
print(crypto_mined_df.shape)
crypto_mined_df.head(10)

(532, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [34]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_coinNames_df = crypto_mined_df[['CoinName']]
print(crypto_coinNames_df.shape)
crypto_coinNames_df.head()

(532, 1)


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [35]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_mined_df.drop(columns=["CoinName"], inplace=True)
print(crypto_mined_df.shape)
crypto_mined_df.head(10)

(532, 4)


C:\Users\sfand\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [36]:
# Check and change dtypes
crypto_mined_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [37]:
crypto_df = crypto_mined_df.astype({'TotalCoinSupply':float})
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07
ETH,Ethash,PoW,1.076842e+08,0.000000e+00


In [38]:
crypto_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
dtype: object

In [39]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df,  columns=["Algorithm","ProofType"])
print(X.shape)
X.head(10)

(532, 98)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,8.400000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,2.200000e+07,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,2.100000e+08,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# Standardize the data with StandardScaler().
crypto_scaled = StandardScaler().fit_transform(X)
crypto_scaled[0:5]

array([[-0.11710817, -0.1528703 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.07530656, -0.0433963 , -0.06142951, -0.06142951,
        -0.0433963 , -0.0433963 , -0.19245009, -0.06142951, -0.09740465,
        -0.0433963 , -0.11547005, -0.07530656, -0.0433963 , -0.0433963 ,
        -0.15191091, -0.0433963 , -0.13118084, -0.0433963 , -0.0433963 ,
        -0.08703883, -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.0433963 , -0.08703883, -0.08703883, -0.08703883,
        -0.0433963 , -0.13118084, -0.13840913, -0.13840913, -0.0433963 ,
        -0.06142951, -0.0433963 , -0.07530656, -0.18168574, -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.07530656, -0.15826614, -0.31491833,
        -0.0433963 , -0.08703883, -0.07530656, -0.06142951,  1.38675049,
        -0.0433963 , -0.0433963 , -0.06142951, -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.39879994, -0.0433963 , -0.1

### Deliverable 2: Reducing Data Dimensions Using PCA

In [41]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)
crypto_pca

array([[-0.32878613,  1.02178694, -0.56367106],
       [-0.31214353,  1.02194598, -0.5641776 ],
       [ 2.31384708,  1.70176486, -0.57092506],
       ...,
       [ 0.33473042, -2.32743215,  0.3646508 ],
       [-0.14328474, -1.99732965,  0.45839656],
       [-0.3015897 ,  0.77482328, -0.30023159]])

In [42]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
            data= crypto_pca, columns=['PC 1', 'PC 2', 'PC 3'], index=crypto_df.index)

print(pcs_df.shape)
pcs_df.head(10) 

(532, 3)


,PC 1,PC 2,PC 3
42,-0.328786,1.021787,-0.563671
404,-0.312144,1.021946,-0.564178
1337,2.313847,1.701765,-0.570925
BTC,-0.131769,-1.290825,0.142418
ETH,-0.146648,-2.047365,0.329418
LTC,-0.161820,-1.156415,-0.055481
DASH,-0.395892,1.291064,-0.421722
XMR,-0.132332,-2.138790,0.464224
ETC,-0.145092,-2.047467,0.329396
ZEC,-0.143284,-1.997330,0.458397


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [43]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

In [44]:
# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

In [45]:
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [46]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)
model

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,

In [47]:
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07
ETH,Ethash,PoW,1.076842e+08,0.000000e+00


In [48]:
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.328786,1.021787,-0.563671
404,-0.312144,1.021946,-0.564178
1337,2.313847,1.701765,-0.570925
BTC,-0.131769,-1.290825,0.142418
ETH,-0.146648,-2.047365,0.329418


In [61]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], ignore_index=False, axis=1, join='inner') #on index
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.328786,1.021787,-0.563671
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.312144,1.021946,-0.564178
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.313847,1.701765,-0.570925
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.131769,-1.290825,0.142418
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.146648,-2.047365,0.329418
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.161820,-1.156415,-0.055481
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.395892,1.291064,-0.421722
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.132332,-2.138790,0.464224
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.145092,-2.047467,0.329396
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.143284,-1.997330,0.458397


In [62]:
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.concat([clustered_df, crypto_coinNames_df], ignore_index=False, axis=1, join='inner')
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.328786,1.021787,-0.563671,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.312144,1.021946,-0.564178,404Coin
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.313847,1.701765,-0.570925,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.131769,-1.290825,0.142418,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.146648,-2.047365,0.329418,Ethereum
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.161820,-1.156415,-0.055481,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.395892,1.291064,-0.421722,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.132332,-2.138790,0.464224,Monero
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.145092,-2.047467,0.329396,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.143284,-1.997330,0.458397,ZCash


In [63]:
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["class"] = model.labels_
clustered_df.head()

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.328786,1.021787,-0.563671,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.312144,1.021946,-0.564178,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.313847,1.701765,-0.570925,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.131769,-1.290825,0.142418,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.146648,-2.047365,0.329418,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,8.400000e+07,-0.161820,-1.156415,-0.055481,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,2.200000e+07,-0.395892,1.291064,-0.421722,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,-0.132332,-2.138790,0.464224,Monero,0
ETC,Ethash,PoW,1.133597e+08,2.100000e+08,-0.145092,-2.047467,0.329396,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,2.100000e+07,-0.143284,-1.997330,0.458397,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [73]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="CoinName",
    y="Algorithm",
    z="ProofType",
    color="class",
    symbol="class",
    width=800,
    hover_name="CoinName",
    #hover_data="Algorithm"
    
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [74]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [95]:
# Print the total number of tradable cryptocurrencies.
print(f'There are {clustered_df.CoinName.count()} tradable cryptocurrencies.')

There are 532 tradable cryptocurrencies.


In [96]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
crypto_minmax = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
crypto_minmax_scaled = MinMaxScaler().fit_transform(crypto_minmax)
crypto_minmax_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [97]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(
            data= crypto_minmax_scaled, columns=['TotalCoinSupply', 'TotalCoinsMined'], index=clustered_df.index)
#plot_df.head(10)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df1 = pd.concat([plot_df, clustered_df['CoinName']], ignore_index=False, axis=1, join='inner')
#plot_df1.head(10)

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df2 = pd.concat([plot_df1, clustered_df['class']], ignore_index=False, axis=1, join='inner')
plot_df2.head(10)

#plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,class
42,4.200000e-11,0.000000,42 Coin,1
404,5.320000e-04,0.001066,404Coin,1
1337,3.141593e-01,0.029576,EliteCoin,1
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,1
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [100]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df2.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="class",
)


:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)